In [1]:
from openai import OpenAI
import time
import re
import json
import os
from dotenv import load_dotenv
from src.LLM_prompting import *
from src.html_generation import * 
import pdfkit 
from src.CvBuilder import CvBuilder
from src.MongoController import MongoController

In [2]:
model_choice = os.getenv("OPENAI_MODEL_CHOICE")
openai_api_key = os.getenv("OPEN_AI_API_KEY")

client = OpenAI(
    api_key=openai_api_key
)
system = {
    "model_choice" :  model_choice,
    'system_prompt' : "You are an helpful assistant, you will only use provided informations to answer and will not assume informations. You need to output only relevant information, introduction message are not required"
}

In [3]:
mongo_controller = MongoController()
builder = CvBuilder(mongo_controller, client, system)

In [12]:
builder.load()

State 67291ab592bcaf074801f8ce loaded for user 672918ed6de6f79da9ef60f5


In [ ]:
with open('../job_description.txt','r') as fd:
    job_description = fd.read()

builder.generate_cv_from_job_description(job_description)


In [6]:
builder.save()

State saved InsertOneResult(ObjectId('67291ab592bcaf074801f8ce'), acknowledged=True)


In [6]:
builder.generate_cv_pdf('../output/cv.pdf')

CV created!


In [5]:

# user_data['static_context']
# user_data['general_info'] 
# user_data['skills']
# user_data['work_experience']
# user_data['work_experience_meta'] 
# user_data['education'] 

{'static_context': {'full_name': 'Gabriel Ludel', 'location': 'Tallinn', 'email': 'gabriel.ludel@gmail.com', 'phone': '<b>+33767534950</b>', 'portfolio_link': 'https://gludel.com'}, 'general_info': "**Professional Background:**\n1. **Roles and Responsibilities**:\n   - Former Lead Fullstack Developer at my previous company.\n   - Managed the complete IT infrastructure for the company.\n   - Built several production applications, including an ETL, CRM, and user applications from scratch.\n   - Experienced in managing the full life cycle of a web application.\n\n2. **Technical Skills**:\n   - Fullstack development (4 years of experience).\n   - Defined application infrastructure using microservices, DevOps, Docker, CI/CD pipelines, and software design patterns.\n   - Experience in DevOps and data engineering, including data analysis.\n   - Knowledge of domain and hosting environments, including setup and management.\n\n3. **Educational Background**:\n   - Bachelor's degree in Computer Sc

---

From json to mongo

In [6]:
with open('./personal_info.json','r') as fd:
    user_data = json.load(fd)

mongo_controller = MongoController()
mongo_controller.save_user_info(user_data)


InsertOneResult(ObjectId('672918ed6de6f79da9ef60f5'), acknowledged=True)

In [14]:

state = {
    'job_infos': builder.job_infos,
    'work_context': builder.work_context,
    'project_context': builder.project_context,
    'relevant_skillset': builder.relevant_skillset,
    'profile_description': builder.profile_description,
    'profile_title': builder.profile_title,
    'job_description' : job_description
}
user_id = builder.user_id

mongo_controller.save_user_cv(user_id,state)

InsertOneResult(ObjectId('672919086de6f79da9ef60f7'), acknowledged=True)